# Innolitics Feature Detection Algorithm

This notebook outlines an approach to detecting the grid intersections in a volume via convolution with a kernel with the same geometry as the known grid intersections.

In other words, we use our knowledge of the phantom's construction to produce a kernel that matches the geometry of the grid intersections.  The kernel could also be modified to account for the effects of the imaging system.

Once we have the kernel, we simply:

1. Convolve it with the data volume
2. Threshold and label each "feature"
3. Find the "center of mass" of the convolution result within each feature

You can see how the algorithm works in the code cells below.

This algorithm has a number of advantages.

- It is soundly based on our a-priori knowledge of the system
- It is very simple; it has very few steps and paramaters, which will make the final result more robust and less heuristical.  I.e. there is only one heuristically driven paramater (the threshold value), and even this paramter can probably be quantified
- It does a good job rejecting false-positives and noise, because it will only select features in the image that match the shape of the grid intersection
- It is robust to slight rotations in the grid; i.e. it should still properly locate the center of the intersection
- Because it does not use different thresholds for each slice, localization along the slice-axis should be just as good as along the non-slice axes; as a result, the algorithm should only require a single data set.

It also has some disadvantages.

- Like Stanescu's algorithm, it won't work if the grid is not rectilinear
- It requires us knowing the phantom we are using (so that we know how to construct the kernel).  This doesn't seem like a problem, as we should expect to know this anyway.  We could also probably estimate the values if we didn't know.
- It is less sensitive to grid poins on the 

In [ ]:
%matplotlib inline

from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

from visualization import slices_interactive, compare_volumes_interactive, compare_volumes, slider, scatter3

def invert(data):
    return 2*np.mean(data) - data

In [ ]:
data = invert(loadmat('../tmp/mri-001-axial-voxels.mat')['voxels'])

In [ ]:
def grid_intersection_kernel(nrx, nry, nrz):
    '''
    Generate a convolution kernel that can be used to detect "grid intersections".
    
    Grid intersections are modeled as the intersections between three cylinders.
    
    The radius of the cylinders along each dimension, nrx, nry, and nrz, are 
    specified in pixels.
    
    The kernel rejects the 0-frequency (i.e. its mean is 0), hence the values 
    outside of the grid will be negative.  In this way, matching features in the
    image will stand out against a background that is approximately 0.
    '''
    kernel_shape = tuple(round(2*4*n) for n in (nrx, nry, nrz))
    kernel = np.zeros(kernel_shape)
    X, Y, Z = np.meshgrid(*(np.linspace(-4, 4, n) for n in kernel_shape), indexing='ij')
    kernel[X**2 + Y**2 < 1] = 1
    kernel[X**2 + Z**2 < 1] = 1
    kernel[Y**2 + Z**2 < 1] = 1
    return kernel - np.mean(kernel[:])
    
kernel = grid_intersection_kernel(3, 3, 3)
slices_interactive(kernel)

In [ ]:
from scipy import signal

kernel = grid_intersection_kernel(3, 3, 3)
intersections = signal.fftconvolve(data, kernel, mode='same')
slices_interactive(intersections)

In [ ]:
intersections_thresholded = intersections > np.max(intersections)*0.6
slices_interactive(intersections_thresholded)

In [ ]:
from scipy import ndimage

labels, number_of_labels = ndimage.label(intersections_thresholded)
slices_interactive(labels)
grid_intersections = ndimage.center_of_mass(data, labels, range(1, number_of_labels + 1))
x, y, z = zip(*grid_intersections)

In [ ]:
%matplotlib notebook
scatter3(x, y, z)